# IEEE39 - Validation against PF

In [ ]:
import subprocess, sys, os
import urllib.request

dpsim_root_dir = subprocess.Popen(['git', 'rev-parse', '--show-toplevel'], stdout=subprocess.PIPE).communicate()[0].rstrip().decode('utf-8')
sys.path.insert(0, os.path.join(dpsim_root_dir, 'python/src/dpsim/'))
sys.path.insert(0, os.path.join(dpsim_root_dir, 'build'))

import matpower
import dpsimpy
from villas.dataprocessing.readtools import *
from villas.dataprocessing.timeseries import *
import urllib.request
import matplotlib.pyplot as plt
import numpy as np

### Get simulation data

In [ ]:
if not os.path.exists('ieee39-data'):
    os.mkdir('ieee39-data')

file_names = ["20140528T0800Z_XX_YYY_DY_.xml", "20140528T0800Z_XX_YYY_SSH_.xml", "20140528T0800Z_XX_YYY_SV_.xml", "20140528T0800Z_XX_YYY_TP_.xml", "20140528T0800Z_YYY_EQ_.xml"]
for file_name in file_names:
    url_dynamic = 'https://raw.githubusercontent.com/martinmoraga/dpsim_data/main/IEEE39/CIM/' + file_name
    local_file = './ieee39-data/' + file_name
    urllib.request.urlretrieve(url_dynamic, local_file)
    
path_to_files = []
for item in file_names: path_to_files.append('./ieee39-data/' + item)

### PF Simulation

In [ ]:
sim_name_pf = "IEEE39"

dpsimpy.Logger.set_log_dir('logs/' + sim_name_pf)
reader = dpsimpy.CIMReader(sim_name_pf, dpsimpy.LogLevel.debug, dpsimpy.LogLevel.debug)
system_pf = reader.loadCIM(60, path_to_files, dpsimpy.Domain.SP, dpsimpy.PhaseType.Single, dpsimpy.GeneratorType.PVNode)
gen_ref = system_pf.component("G 02")
gen_ref.modify_power_flow_bus_type(dpsimpy.PowerflowBusType.VD)

logger = dpsimpy.Logger(sim_name_pf)
for node in system_pf.nodes:
    logger.log_attribute(node.name() + '.V', 'v', node)
    logger.log_attribute(node.name() + '.S', 's', node)
    
sim = dpsimpy.Simulation(sim_name_pf, dpsimpy.LogLevel.debug)
sim.set_system(system_pf)
sim.set_time_step(0.1)
sim.set_final_time(0.1)
sim.set_domain(dpsimpy.Domain.SP)
sim.set_solver(dpsimpy.Solver.NRP)
sim.do_init_from_nodes_and_terminals(True)

sim.add_logger(logger)
sim.run()

### Read pf results

In [ ]:
dpsim_result_file = 'logs/' + sim_name_pf + '/' + sim_name_pf + '.csv'
ts_dpsim_pf = read_timeseries_csv(dpsim_result_file)

### DPsim Results

In [ ]:
dpsim_results = pd.DataFrame(columns=['Bus', 'Vmag [kV]', 'Va [°]', 'P [MW]', 'Q [MVAr]'])
w_mw = 1e-6
v_kV = 1e-3
for i in range (39):
    node_name = f"Bus {i+1:02d}"
    dpsim_results.loc[i] = (
        [node_name] 
        + [np.round(np.absolute(ts_dpsim_pf[node_name + '.V'].values[-1] * v_kV), 4)]
        + [np.round(np.angle(ts_dpsim_pf[node_name + '.V'].values[-1]) * 180/np.pi , 4)] 
        + [np.round(np.real(ts_dpsim_pf[node_name + '.S'].values[-1] * w_mw), 4)] 
        + [np.round(np.imag(ts_dpsim_pf[node_name + '.S'].values[-1] * w_mw), 4)])

dpsim_results

### DigSilent Results

In [ ]:
res = reader.get_powerflow_results()
pf_results = pd.DataFrame(columns=['Bus', 'Vmag [kV]', 'Va [°]', 'P [MW]', 'Q [MVAr]'])
w_mw = 1e-6
v_kV = 1
for node_name, data in res.items():
    loc = int(node_name[4:]) - 1
    pf_results.loc[loc] = (
        [node_name] 
        + [round(data[0] * v_kV, 4)]
        + [round(data[1], 4)] 
        + [round(data[2], 4)] 
        + [round(data[3], 4)])

pf_results

In [ ]:
difference_vmag = pf_results['Vmag [kV]'] - dpsim_results['Vmag [kV]']
difference_vangle = pf_results['Va [°]'] - dpsim_results['Va [°]']

#base power = 100MW
difference_p = (pf_results['P [MW]'] - dpsim_results['P [MW]'])
difference_q = (pf_results['Q [MVAr]'] - dpsim_results['Q [MVAr]'])
difference = {"Bus": pf_results['Bus'], "Vmag [kV]": difference_vmag, "Va [°]": difference_vangle, "P [MW]": difference_p, "Q [MVAr]": difference_q}
difference = pd.DataFrame(difference)

difference
